In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_list = ['./110S1/', './110S2/', './110S3/', './110S4/', './111S1/', './111S2/', './111S3/', './111S4/']
dfs = []

for file in file_list:
    df_temp = pd.read_csv(file + 'A_lvr_land_A.csv')
    dfs.append(df_temp)

# 將資料串接起來
dfA = pd.concat(dfs)

In [ ]:
dfA

In [ ]:
dfsb = []

for file in file_list:
    df_temp = pd.read_csv(file + 'A_lvr_land_B.csv')
    dfsb.append(df_temp)

# 將資料串接起來
dfB = pd.concat(dfsb)

In [ ]:
dfB

In [ ]:
# 把欄位名稱修改一致
dfA.rename(columns={'車位移轉總面積(平方公尺)': '車位移轉總面積平方公尺'}, inplace=True)

# 增加是否為預售屋的欄位
dfA['預售屋'] = '0'
dfB['預售屋'] = '1'

In [ ]:
# 移除dfA和dfB的第一個row（英文名稱）
dfA = dfA.drop(dfA.index[0])
dfB = dfB.drop(dfB.index[0])

# 合併一般不動產和預售屋的資料
df_merged = pd.concat([dfA, dfB])

In [ ]:
df_merged.columns

In [ ]:
df_merged.head(3)

## Exploratory Data Analysis

In [ ]:
# from matplotlib.font_manager import fontManager

# for i in sorted(fontManager.get_font_names()):
#     print(i)

In [ ]:
# import matplotlib.pyplot as plt

# # plt.rcParams['font.sans-serif'] = ['SimHei']  # 替換成您安裝的中文字型

# df_merged['鄉鎮市區'].value_counts().plot(kind='bar')

# # 設定圖表標題和軸標籤
# plt.title('鄉鎮市區資料數統計')
# plt.xlabel('鄉鎮市區')
# plt.ylabel('數量')

# # 顯示圖表
# plt.show()

## Data Cleaning

In [ ]:
# 移除不需要的欄位
df_merged = df_merged.drop(['交易標的', '都市土地使用分區', '非都市土地使用分區', '非都市土地使用編定', '總樓層數', '主要用途',
              '主要建材', '車位總價元', '編號', '移轉編號', '建案名稱', '棟及號'], axis=1)

In [ ]:
df_merged.head(3)

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 將建築完成年月改成建築完成幾年
df_merged['建築完成距今'] = df_merged['建築完成年月'].apply(lambda x: datetime.strptime(str(int(x) + 19110000), '%Y%m%d').date() if pd.notnull(x) and isinstance(x, (int, str)) and len(str(x)) in [6, 7, 8] else None)

# 計算交易距今的相差年數
current_date = datetime.now().date()  # 獲取當前日期
df_merged['建築完成距今'] = df_merged['建築完成距今'].apply(lambda x: relativedelta(current_date, x).years)  # 計算相差的年數

In [ ]:
df_merged

In [ ]:
# 計算每個鄉鎮市區的建築完成距今的平均值
average_by_district = df_merged.groupby('鄉鎮市區')['建築完成距今'].mean()

# 依據鄉鎮市區填入建築完成距今為0的資料的平均值
df_merged['建築完成距今'] = df_merged.apply(lambda row: average_by_district[row['鄉鎮市區']] if row['建築完成距今'] == 0 else row['建築完成距今'], axis=1)
df_merged.loc[df_merged['預售屋'] == 1, '建築完成距今'] = 0

In [ ]:
df_merged.drop('建築完成年月', axis=1, inplace=True)

In [ ]:
df_merged.head(3)

In [ ]:
# 把鄉鎮市區變成dummy variables
df_merged = pd.get_dummies(df_merged, columns=['鄉鎮市區'])
df_merged.head(3)

In [ ]:
# 將"交易筆棟數"拆成3個欄位
df_merged[['交易土地', '交易建物', '交易車位']] = df_merged['交易筆棟數'].str.extract(r'土地(\d+)建物(\d+)車位(\d+)')

# 將數字轉換為整數型態
df_merged[['交易土地', '交易建物', '交易車位']] = df_merged[['交易土地', '交易建物', '交易車位']].astype(int)

# 移除原始的「交易筆棟數」欄位
df_merged.drop('交易筆棟數', axis=1, inplace=True)

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# 將交易年月日改成交易完成幾年
df_merged['交易年月日'] = df_merged['交易年月日'].apply(lambda x: datetime.strptime(str(int(x)+19110000), '%Y%m%d'))  # 將字串轉換為日期格式

# 計算交易距今的相差年數
current_date = datetime.now().date()  # 獲取當前日期
df_merged['交易距今'] = df_merged['交易年月日'].apply(lambda x: relativedelta(current_date, x).years)  # 計算相差的年數

# 移除原始的「交易年月日」欄位
df_merged.drop('交易年月日', axis=1, inplace=True)

In [ ]:
df_merged.head(3)

In [ ]:
import re

def extract_street(address):
    # 定義街道名稱的正則表達式
    pattern = r"(?P<city>[\u4e00-\u9fa5]+市)?(?P<district>[\u4e00-\u9fa5]+[區市里])?(?P<street>[\u4e00-\u9fa5]+[路街巷道段]).*"

    # 使用正則表達式進行匹配
    match = re.match(pattern, address)
    
    if match:
        street_name = match.group("street")
        
        # 去除市區、巷、弄、號等元素
        street_name = re.sub(r"[市區里].*$", "", street_name)
        street_name = re.sub(r"[巷弄號].*$", "", street_name)
        return street_name
    else:
        return None

df_merged['路段'] = df_merged['土地位置建物門牌'].apply(extract_street)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 創建 LabelEncoder 物件
label_encoder = LabelEncoder()

# 將類別變數進行編碼
df_merged['路段'] = label_encoder.fit_transform(df_merged['路段'])

# 刪除土地位置建物門牌
df_merged.drop('土地位置建物門牌', axis=1, inplace=True)

In [ ]:
df_merged.head(3)

In [ ]:
# 查看 "單價元平方公尺" 欄位的空值數量
null_count = df_merged['單價元平方公尺'].isnull().sum()

# 輸出結果
print("單價元平方公尺的空值數量：", null_count)
print("單價元平方公尺的空值比例：", null_count/58257)

In [ ]:
# 移除 "單價元平方公尺" 欄位為空值的資料列
df_merged.dropna(subset=['單價元平方公尺'], inplace=True)

# 輸出結果
print("移除空值後的資料筆數：", len(df_merged))

In [ ]:
# 移除不需要的欄位
df_merged = df_merged.drop(['土地移轉總面積平方公尺', '建物移轉總面積平方公尺', '總價元', '車位移轉總面積平方公尺',
               '備註', '主建物面積', '附屬建物面積', '陽台面積'], axis=1)

In [ ]:
# 建立層次字串映射的字典
layer_mapping = {
    '地下一層': -1, '地下二層': -2, '地下三層': -3, '地下四層': -4, '地下五層': -5,
    '一層': 1, '二層': 2, '三層': 3, '四層': 4, '五層': 5,
    '六層': 6, '七層': 7, '八層': 8, '九層': 9, '十層': 10,
    '十一層': 11, '十二層': 12, '十三層': 13, '十四層': 14, '十五層': 15,
    '十六層': 16, '十七層': 17, '十八層': 18, '十九層': 19, '二十層': 20,
    '二十一層': 21, '二十二層': 22, '二十三層': 23, '二十四層': 24, '二十五層': 25, 
    '二十六層': 26, '二十七層': 27, '二十八層': 28, '二十九層': 29, '三十層': 30,
}

def extract_number(text):
    if isinstance(text, str):
        for number in sorted(layer_mapping.keys(), key=len, reverse=True):
            match = re.search(number, text)
            if match:
                return layer_mapping[number]
    return 0

# 將 '移轉層次' 欄位中的中文數字轉換為阿拉伯數字
df_merged['移轉層次-數字'] = df_merged['移轉層次'].apply(extract_number)


In [ ]:
df_merged.drop('移轉層次', axis=1, inplace=True)

In [ ]:
df_merged

In [ ]:
import numpy as np

# 填充空值
df_merged['電梯'] = df_merged['電梯'].fillna('未知')

# 條件判斷並填值
df_merged['電梯'] = np.where(
    (df_merged['電梯'] == '未知') & 
    (df_merged['建物型態'].isin(['住宅大樓(11層含以上有電梯)', '華廈(10層含以下有電梯)', '辦公商業大樓'])),
    1,
    0
)

In [ ]:
# 把建物型態變成dummy variables
df_merged = pd.get_dummies(df_merged, columns=['建物型態'])
df_merged.head(3)

In [ ]:
df_merged['建物現況格局-隔間'] = df_merged['建物現況格局-隔間'].replace({'有': 1, '無': 0})
df_merged['有無管理組織'] = df_merged['有無管理組織'].replace({'有': 1, '無': 0})

In [ ]:
df_merged

In [ ]:
# 查看 "單價元平方公尺" 欄位的空值數量
null_count = df_merged['車位類別'].isnull().sum()

# 輸出結果
print("車位類別的空值數量：", null_count)
# print("單價元平方公尺的空值比例：", null_count/58257)

In [ ]:
df_merged.drop('車位類別', axis=1, inplace=True)

In [ ]:
df_merged

In [ ]:
df_merged.to_csv('df_merged.csv', index=False)